In [ ]:
COHORT = 'MY_COHORT'

# Setup - Run following cell, restart kernel, comment following cell

In [ ]:
# This only needs to be run once. Feel free to comment after first run
# Delete this cell after https://broadworkbench.atlassian.net/browse/IA-1402 is fixed.
!pip2.7 install --upgrade pandas-gbq
!pip3 install --upgrade pandas-gbq

In [ ]:
import firecloud.api as fapi
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype
import pandas as pd
import pandas_gbq

import os
WS_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WS_NAME = os.environ['WORKSPACE_NAME']

# Get cohort SQL query

In [ ]:
cohort_query = fapi.get_entity(WS_NAMESPACE, WS_NAME, 'cohort', COHORT).json()['attributes']['query']
cohort_query

# Create pandas dataframe of cohort participant ids

In [ ]:
cohort_participant_ids = pd.read_gbq(cohort_query, dialect='standard')
cohort_participant_ids.head()

# See what tables are available to join against

In [ ]:
dataset_name = fapi.get_entity(WS_NAMESPACE, WS_NAME, 'cohort', COHORT).json()['attributes']['dataset_name']
bigquery_table_entities_all_datasets = fapi.get_entities(WS_NAMESPACE, WS_NAME, 'BigQuery_table').json()
bigquery_table_entities = list(filter(lambda entity: entity['attributes']['dataset_name'] == dataset_name, bigquery_table_entities_all_datasets))
bigquery_tables = list(map(lambda entity: entity['attributes']['table_name'], bigquery_table_entities))
bigquery_tables

# Join cohort participant ids against first table

In [ ]:
table = pd.read_gbq("SELECT * FROM `{}`".format(bigquery_tables[0]), dialect="standard")
print("table has %d rows" % len(table.index))

cohort = cohort_participant_ids.join(table, lsuffix='_L', rsuffix='_R')
print("cohort has %d rows\n" % len(cohort.index))

cohort.head()

# Visualization of cohort first column

In [ ]:
plt.rcParams.update({'font.size': 14})

# Drop columns with "ID" or "id"
cohort_to_plot = cohort[cohort.columns.drop(list(cohort.filter(regex='id|ID')))]
cohort_first_column = cohort_to_plot[cohort_to_plot.columns[0]]

title = '{} for cohort {}'.format(cohort_first_column.name, COHORT)
if is_numeric_dtype(cohort_first_column):
    cohort_first_column.plot(kind='hist', title=title)
else:
    cohort_first_column.value_counts().plot(kind='bar', title=title)

# Provenance

In [ ]:
from datetime import datetime
from pytz import timezone

zone = timezone('US/Eastern')
# zone = timezone('US/Pacific')
print(datetime.now(zone).strftime('%Y-%m-%d %H:%M:%S'))